LeFFA
===

**Learning Flow Fields in Attention for Controllable Person Image Generation**

 * Paper: https://arxiv.org/abs/2412.08486


![Leffa Overview](../assets/leffa_overview.png)




```bash
git clone https://github.com/franciszzj/Leffa.git
cd Leffa/


conda create -n leffa python==3.10
conda activate leffa
pip install -r requirements.txt 
pip install ipykernel
python -m ipykernel install --user --name leffa --display-name "leffa"
```

In [2]:
import sys
sys.path.append("Leffa_repo")
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from huggingface_hub import snapshot_download

from leffa.transform import LeffaTransform
from leffa.model import LeffaModel
from leffa.inference import LeffaInference
from leffa_utils.garment_agnostic_mask_predictor import AutoMasker
from leffa_utils.densepose_predictor import DensePosePredictor
from leffa_utils.utils import resize_and_center, list_dir, get_agnostic_mask_hd, get_agnostic_mask_dc, preprocess_garment_image
from preprocess.humanparsing.run_parsing import Parsing
from preprocess.openpose.run_openpose import OpenPose

## Download checkpoints

In [3]:
# Download checkpoints
snapshot_download(repo_id="franciszzj/Leffa", local_dir="./ckpts")

Fetching 37 files: 100%|██████████| 37/37 [00:00<00:00, 2629.03it/s]


'/home/pyml/git_vmirly/GenAI-Hub/tmp/Leffa/ckpts'